Hiring Hackathon - https://datahack.analyticsvidhya.com/contest/job-a-thon-june-2021
<p><b>Problem Statement: </b>Your Client ComZ is an ecommerce company. The company wants to focus on targeting the right customers  with the right products to increase overall revenue and conversion rate.

To target the right customers with the right products, they need to build an ML model for marketing based on user interaction with products in the past like number of views,  most viewed product, number of activities of user, vintage of user and others.
ComZ has contacted the Data Science and Engineering team to use this information to fuel the personalized advertisements, email marketing campaigns, or special offers on the landing and category pages of the company's website.

You, being a part of the data engineering team, are expected to “Develop input features”  for the efficient marketing model given the Visitor log data and User Data.</p>

In [1]:
#import necessary libraries

import pandas as pd 
import numpy as np
from datetime import datetime
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [2]:
#date for calculation

lastdayfrom = pd.to_datetime('5/28/2018') # end of the record

In [3]:
#create dataframe with visitor Log Data 

visitor_log_data = pd.read_csv('D:\\Hackathon\\DataEngineering\\data\\VisitorLogsData.csv') 

In [4]:
#create dataframe with user data 

user_data = pd.read_csv('D:\\Hackathon\\DataEngineering\\data\\userTable.csv')

In [5]:
#input feature dataframe initialize

input_feature = pd.read_csv('D:\\Hackathon\\DataEngineering\\sample_submission_M7Vpb9f.csv')


In [6]:
visitor_log_data.head(5)

,webClientID,VisitDateTime,ProductID,UserID,Activity,Browser,OS,City,Country
0,WI10000050298,2018-05-07 04:28:45.970,pr100631,NaN,NaN,Chrome Mobile,Android,Chennai,India
1,WI10000025922,2018-05-13 07:26:04.964,pr100707,NaN,NaN,Chrome,Windows,NaN,Taiwan
2,WI100000204522,2018-05-11 11:43:42.832,pr100030,NaN,click,Chrome,windows,Gurgaon,India
3,WI10000011974,2018-05-13 15:20:23.436,Pr100192,NaN,CLICK,Chrome,Windows,,
4,WI100000441953,2018-05-08 20:44:25.238,Pr100762,NaN,click,Chrome,mac os x,Iselin,United States


In [7]:
user_data.head()

,UserID,Signup Date,User Segment
0,U133159,2018-04-14 07:01:16.202607+00:00,C
1,U129368,2017-12-02 09:38:41.584270+00:00,B
2,U109654,2013-03-19 11:38:55+00:00,B
3,U108998,2018-01-18 08:29:51.627954+00:00,C
4,U131393,2018-03-27 08:05:28.806800+00:00,B


In [8]:
print('Size of VisitorLogsData',visitor_log_data.shape)
print('Size of userTable',user_data.shape)

Size of VisitorLogsData (6588000, 9)
Size of userTable (34050, 3)


In [9]:
#null present in the visitor log data

visitor_log_data.isnull().sum()

webClientID            0
VisitDateTime     658915
ProductID         527137
UserID           5937305
Activity          889446
Browser                0
OS                     0
City             2165831
Country           397693
dtype: int64

In [10]:
#null in user data

user_data.isnull().sum()

UserID          0
Signup Date     0
User Segment    0
dtype: int64

## Clean Dataset

In [11]:
#field City and Country in visitor_log_data is not necessary so drop them

visitor_log_data.drop(['City','Country','webClientID'],axis=1,inplace=True)

In [12]:
#drop the rows where UserID is NULL

print('Previous DF size', len(visitor_log_data))
visitor_log_data.dropna(subset=['UserID'],inplace=True)
print('Post DF size', len(visitor_log_data))

Previous DF size 6588000
Post DF size 650695


In [13]:
# visitor_log_data[visitor_log_data['UserID']=='U115735'].head(30)

In [14]:
#sorted the dataframe by UserID and then ffill VisitDateTime

visitor_log_data.sort_values(by=['UserID','ProductID'],inplace = True)
visitor_log_data['VisitDateTime'].ffill(inplace=True)
visitor_log_data.sort_index(inplace=True)

In [15]:
#transfer unix date time to normal


visitor_log_data['VisitDateTime'] = visitor_log_data['VisitDateTime'].apply(lambda x: pd.to_datetime(int(x)) 
                                                                              if ('-' not in str(x)) else x)

In [16]:
#first we sort the data based on 'UserID','VisitDateTime' and then we ffill the NULLs of Activity

visitor_log_data.sort_values(by=['UserID','VisitDateTime'],inplace = True)
visitor_log_data['Activity'].ffill(inplace=True)
visitor_log_data['ProductID'].bfill(inplace=True)
visitor_log_data.sort_index(inplace=True)

In [17]:
#change strings to upper case for uniformity

visitor_log_data['Activity'] = visitor_log_data['Activity'].str.upper()
visitor_log_data['Browser'] = visitor_log_data['Browser'].str.upper()
visitor_log_data['OS'] = visitor_log_data['OS'].str.upper()
visitor_log_data['ProductID'] = visitor_log_data['ProductID'].str.upper()

# Extract Features

#### Does the data has only 21days record?

In [18]:
len(visitor_log_data) == len(visitor_log_data[visitor_log_data['VisitDateTime'] >= (pd.to_datetime(lastdayfrom) - pd.Timedelta(days=21))])

True

## No_of_days_Visited_7_Days

How many days a user was active on platform in the last 7 days.

In [19]:
#dataframe for last 7days data

last_seven_days = visitor_log_data[visitor_log_data['VisitDateTime'] >= (pd.to_datetime(lastdayfrom) - pd.Timedelta(days=7))]

#extract day of date

last_seven_days['days'] = last_seven_days['VisitDateTime'].apply(lambda x: x.day)

In [20]:
last_seven_days['UserID'].nunique() #no of users active. so there are few users who are active in last 7 days

16472

In [21]:
# create dictionary with userid and How many days a user was active on platform in the last 7 days

temp_7_active_usr=last_seven_days[['UserID','days']].groupby(['UserID','days']).count().reset_index()
df_7_active_usr = temp_7_active_usr.groupby('UserID').count().reset_index()
dic_map_usr_7_Days = pd.Series(df_7_active_usr['days'].values,index=df_7_active_usr['UserID']).to_dict()

In [22]:
#filled input_feature mapped with user active in 7 days

input_feature['No_of_days_Visited_7_Days']= input_feature['UserID'].map(dic_map_usr_7_Days)
input_feature['No_of_days_Visited_7_Days'].unique()

array([nan,  1.,  6.,  4.,  2.,  3.,  5.,  7.])

In [23]:
input_feature['No_of_days_Visited_7_Days'].fillna(0,inplace=True) #those user who were not active put zero for them

## No_Of_Products_Viewed_15_Days

Number of Products viewed by the user in the last 15 days

In [24]:
#dataframe for last 7days data

last_15_days = visitor_log_data[visitor_log_data['VisitDateTime'] >= (pd.to_datetime(lastdayfrom) - pd.Timedelta(days=15))]


In [25]:
# create dictionary with userid and How many product a user viewed on platform in the last 15 days

temp_15days_prodct = last_15_days[['UserID','ProductID']].groupby(['UserID','ProductID']).count().reset_index()
df_15days_product = temp_15days_prodct.groupby('UserID')['ProductID'].count().reset_index()
dic_map_15_Days_product = pd.Series(df_15days_product['ProductID'].values,index=df_15days_product['UserID']).to_dict()

In [26]:
#filled input_feature mapped with user active in 15 days

input_feature['No_Of_Products_Viewed_15_Days']= input_feature['UserID'].map(dic_map_15_Days_product)
input_feature['No_Of_Products_Viewed_15_Days'].unique()

array([  2.,   3.,  16.,   1.,  nan,  19.,  13.,   6.,   4.,   5.,   8.,
        42.,  12.,   9.,  91.,  11.,  10.,  31.,  17.,   7.,  35.,  14.,
        29.,  20.,  15.,  28.,  30.,  27.,  26.,  23.,  50.,  68.,  18.,
        24.,  21.,  32., 104.,  47.,  40.,  41.,  22.,  59.,  43.,  33.,
        25.,  39.,  44.,  51.,  37.,  65.,  45.,  34.,  70.,  36.,  52.,
        38.,  78.,  80.,  69.])

In [27]:
#those user who were not viewed products put zero for them

input_feature['No_Of_Products_Viewed_15_Days'].fillna(0,inplace=True) 

## User_Vintage
Vintage (In Days) of the user as of today

In [28]:
#timezone must be excluded from Signup Date to get total Vintage (In Days), then we need only date transformation

user_data['User_Vintage']= (lastdayfrom - pd.to_datetime(user_data['Signup Date']).dt.tz_localize(None))
user_data['User_Vintage'] = user_data['User_Vintage'].dt.days

In [29]:
#created dictionary to map it with input_feature's UserID

user_vinta_dic = pd.Series(user_data['User_Vintage'].values,index=user_data['UserID']).to_dict()
input_feature['User_Vintage']= input_feature['UserID'].map(user_vinta_dic)

## Most_Viewed_product_15_Days
Most frequently viewed (page loads) product by the user in the last 15 days. If there are multiple products that have a similar number of page loads then , consider the recent one. If a user has not viewed any product in the last 15 days then put it as Product101. 

In [30]:
view15_product = last_15_days[last_15_days['Activity'] == 'PAGELOAD'] #filter with PAGELOAD

In [31]:
# in 15 days which products are viewed how many time by user

temp_most_viewd = view15_product[['UserID','ProductID','Activity']].groupby(['UserID','ProductID'])['Activity'].count().reset_index()

In [32]:
#most viewed product

most_viewd_15=temp_most_viewd.iloc[temp_most_viewd.groupby('UserID').apply(lambda x: x['Activity'].idxmax())]


In [33]:
most_viewed_prdct_dic = pd.Series(most_viewd_15['ProductID'].values,index=most_viewd_15['UserID']).to_dict()
input_feature['Most_Viewed_product_15_Days']= input_feature['UserID'].map(most_viewed_prdct_dic)

In [34]:
input_feature['Most_Viewed_product_15_Days'].isna().sum() #null are present 

10089

In [35]:
#nulls will fill up with Product101

input_feature['Most_Viewed_product_15_Days'].fillna('Product101',inplace=True)

## Most_Active_OS
Most Frequently used OS by user

In [36]:
active_os_temp = visitor_log_data[['UserID','OS','Browser']].groupby(['UserID','OS'])['Browser'].count().reset_index()

#most active os

active_os=active_os_temp.iloc[active_os_temp.groupby('UserID').apply(lambda x: x['Browser'].idxmax())]

#create dic to map

active_os_dic = pd.Series(active_os['OS'].values,index=active_os['UserID']).to_dict()
input_feature['Most_Active_OS']= input_feature['UserID'].map(active_os_dic)

## Recently_Viewed_Product

Most recently viewed (page loads) product by the user.
If a user has not viewed any product then put it as Product101.

In [37]:
#recently viewed products
recently_viewed_products = visitor_log_data[['UserID','ProductID','VisitDateTime']][visitor_log_data['Activity']=='PAGELOAD'].groupby(['UserID','ProductID'])['VisitDateTime'].agg('max').reset_index()

#most recent view product
temp_recent_prodct = recently_viewed_products.iloc[recently_viewed_products.groupby('UserID').apply(lambda x: x['VisitDateTime'].idxmax())]

#create dic to map
temp_recent_prodct_dic = pd.Series(temp_recent_prodct['ProductID'].values,index=temp_recent_prodct['UserID']).to_dict()
input_feature['Recently_Viewed_Product']= input_feature['UserID'].map(temp_recent_prodct_dic)

In [38]:
#fillup the nan

input_feature['Recently_Viewed_Product'].fillna('Product101',inplace=True)

## Pageloads_last_7_days

Count of Page loads in the last 7 days by the user

In [39]:
Pageloads_last_7 = last_seven_days[last_seven_days['Activity']=='PAGELOAD'].groupby('UserID')['Activity'].count().reset_index()
Pageloads_last_7_dic = pd.Series(Pageloads_last_7['Activity'].values,index=Pageloads_last_7['UserID']).to_dict()
input_feature['Pageloads_last_7_days']= input_feature['UserID'].map(Pageloads_last_7_dic)

In [40]:
#fill null with 0

input_feature['Pageloads_last_7_days'].fillna(0,inplace=True)

## Clicks_last_7_days
Count of Clicks in the last 7 days  by the user

In [41]:
CLICK_last_7 = last_seven_days[last_seven_days['Activity']=='CLICK'].groupby('UserID')['Activity'].count().reset_index()
CLICK_last_7_dic = pd.Series(CLICK_last_7['Activity'].values,index=CLICK_last_7['UserID']).to_dict()
input_feature['Clicks_last_7_days']= input_feature['UserID'].map(CLICK_last_7_dic)

In [42]:
#fill null with 0

input_feature['Clicks_last_7_days'].fillna(0,inplace=True)

## file creation

In [43]:
input_feature.isna().sum()

UserID                           0
No_of_days_Visited_7_Days        0
No_Of_Products_Viewed_15_Days    0
User_Vintage                     0
Most_Viewed_product_15_Days      0
Most_Active_OS                   0
Recently_Viewed_Product          0
Pageloads_last_7_days            0
Clicks_last_7_days               0
dtype: int64

In [44]:
len(input_feature)

34050

In [45]:
input_feature.to_csv('input20.csv',index=None)